**Random Forest: 2012-2016 training set**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import math
import seaborn as sns
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

The data was provided and it contains stock market data fo approximately 1500 securities listed on the US stock exchange. We'll work with daily data from February 2018 to July 2017. 

In [2]:
data=pd.read_csv("./final_project_data.csv")
data=data.dropna()
data.index=range(len(data))

In [3]:
rf= RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample")
print(rf.get_params())

{'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': 99, 'verbose': 0, 'warm_start': False}


In [4]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 45, stop = 105, num = 15)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 8, num = 5)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

#Grid
random_grid = {'n_estimators': n_estimators,'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [5]:
#Experiment for the first training set: 2008-2012
i=2016
train_rf_5=data[(data.date<=int(str(i)+'1231'))&(data.date>=int(str(i-4)+'0101'))]
train_rf_5=train_rf_5.dropna()
X_train_rf_5=np.asarray(train_rf_5.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_train_rf_5=np.asarray(train_rf_5.loc[:,'target'])

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 5, verbose=2, random_state=42, n_jobs = -1, pre_dispatch=500)
rf_random.fit(X_train_rf_5, Y_train_rf_5)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed: 76.8min remaining: 125.4min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed: 169.5min remaining: 18.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 181.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=99, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
          pre_dispatch=500, random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [6]:
rf_random.best_params_

{'n_estimators': 62,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 8}

Checking accuracy

In [8]:
test_rf_5=data[(data.date<=int(str(i+1)+'1231'))&(data.date>=int(str(i+1)+'0101'))]
test_rf_5=test_rf_5.dropna()
X_test_rf_5=np.asarray(test_rf_5.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_test_rf_5=np.asarray(test_rf_5.loc[:,'target'])

acc_os=1.0*(rf_random.best_estimator_.predict(X_test_rf_5)==np.asarray(Y_test_rf_5)).sum()/len(Y_test_rf_5)
acc_in=1.0*(rf_random.best_estimator_.predict(X_train_rf_5)==np.asarray(Y_train_rf_5)).sum()/len(Y_train_rf_5)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 50.80229409632646
In sample accuracy: 53.93074760498679


In [13]:
X_train_,X_vali_,Y_train_,Y_vali_ = train_test_split(X_train_rf_5, Y_train_rf_5, test_size=0.1, random_state=99)

best_score = 0
n_estimators = [int(x) for x in np.linspace(start = 60, stop = 70, num = 7)]
max_depth = [int(x) for x in np.linspace(7, 9, num = 3)]
for n in n_estimators:
    for d in max_depth:
        clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators= n,
        min_samples_split= 10, min_samples_leaf= 2, max_features='sqrt', max_depth=d, n_jobs=-1)
        clf.fit(X_train_,Y_train_) 
        score = clf.score(X_vali_, Y_vali_)
        if score > best_score:
            best_score = score
        print ("Score:", score, "n_estimator= ", n, "Max_depth=",d )

Score: 0.5244316353102964 n_estimator=  60 Max_depth= 7
Score: 0.5268016502326126 n_estimator=  60 Max_depth= 8
Score: 0.529157035433186 n_estimator=  60 Max_depth= 9
Score: 0.524529166788581 n_estimator=  61 Max_depth= 7
Score: 0.5268991817108972 n_estimator=  61 Max_depth= 8
Score: 0.5295081487550107 n_estimator=  61 Max_depth= 9
Score: 0.5247632423364641 n_estimator=  63 Max_depth= 7
Score: 0.5268991817108972 n_estimator=  63 Max_depth= 8
Score: 0.5300543250334045 n_estimator=  63 Max_depth= 9
Score: 0.5247096000234076 n_estimator=  65 Max_depth= 7
Score: 0.527128380684866 n_estimator=  65 Max_depth= 8
Score: 0.5300104358681764 n_estimator=  65 Max_depth= 9
Score: 0.5248754035364914 n_estimator=  66 Max_depth= 7
Score: 0.5272502950327218 n_estimator=  66 Max_depth= 8
Score: 0.5297666071724649 n_estimator=  66 Max_depth= 9
Score: 0.524992441310433 n_estimator=  68 Max_depth= 7
Score: 0.5273039373457783 n_estimator=  68 Max_depth= 8
Score: 0.5297178414333226 n_estimator=  68 Max_depth

In [14]:
clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators= 63,
min_samples_split= 10, min_samples_leaf= 2, max_features='sqrt', max_depth=9, n_jobs=-1)
clf.fit(X_train_rf_5,Y_train_rf_5) 

acc_os=1.0*(clf.predict(X_test_rf_5)==np.asarray(Y_test_rf_5)).sum()/len(X_test_rf_5)
acc_in=1.0*(clf.predict(X_train_rf_5)==np.asarray(Y_train_rf_5)).sum()/len(Y_train_rf_5)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 51.07263927680532
In sample accuracy: 54.766740709494464
